In [ ]:
!pip install llm2vec
# !pip install -U "transformers" "huggingface_hub" "tokenizers"
!pip install Levenshtein
!pip install "protobuf==3.20.3" --force-reinstall

In [ ]:
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import os, math, pandas as pd, numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch
import Levenshtein
from typing import List, Sequence, Tuple
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import spearmanr, pearsonr
import editdistance
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from collections import deque
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.cuda.amp import autocast, GradScaler
import kagglehub
import random
from scipy.stats import spearmanr

from types import SimpleNamespace
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)


In [ ]:
TASK_TYPE = "pair_cls"
MAX_LENGTH = 256
BATCH_SIZE = 16
epochs = 5
LR = 2e-5
warmup_ratio = 0.06
save_dir = 'ckpt1'
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

teacher_special="<|embed|>"
student_special="[CLS]"
###



In [ ]:
class TextPairRaw(Dataset):
    def __init__(self, df: pd.DataFrame, task: str):
        self.task = task
        if task == "single_cls":
            self.samples = [(t, None, int(y)) for t, y in zip(df["text"].astype(str), df["label"].astype(int))]
        elif task == "pair_cls":
            self.samples = [(a, b) for a,b in zip(df["premise"].astype(str),
                                                            df["hypothesis"].astype(str))]
        else:  # pair_reg
            self.samples = [(a, b) for a,b in zip(df["sentence1"].astype(str),
                                                              df["sentence2"].astype(str))]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx] 
from typing import List, Tuple, Optional



class DualTokenizerCollate:
    """Collate that tokenizes with two tokenizers (student/teacher).

    Span-MinED upgrade: if a tokenizer is *fast*, we also return `offset_mapping`
    so attention alignment can be done via character-span overlap (robust 1-to-many).
    For slow tokenizers, we simply omit offsets (and downstream code falls back).
    """
    def __init__(self, tok_student, tok_teacher, task: str, max_len: int):
        self.ts = tok_student
        self.tt = tok_teacher
        self.task = task
        self.max_len = max_len

        self._stu_fast = bool(getattr(tok_student, "is_fast", False))
        self._tea_fast = bool(getattr(tok_teacher, "is_fast", False))

    def _encode(self, tok, texts, want_offsets: bool):
        kwargs = dict(
            max_length=self.max_len,
            truncation=True,
            padding=True,
            return_tensors="pt",
            return_special_tokens_mask=True,
        )
        # `return_offsets_mapping` is only supported by *fast* tokenizers.
        if want_offsets:
            kwargs["return_offsets_mapping"] = True
        return tok(list(texts), **kwargs)

    def __call__(self, batch):
        # Supported batch formats:
        # - single_cls: (text, None, label)
        # - pair_*: (s1, s2)
        if self.task == "single_cls":
            texts, labels = [], []
            for item in batch:
                # tolerate both (text, y) and (text, _, y)
                if len(item) == 2:
                    t, y = item
                else:
                    t, _, y = item
                texts.append(t)
                labels.append(int(y))

            s_enc = self._encode(self.ts, texts, self._stu_fast)
            t_enc = self._encode(self.tt, texts, self._tea_fast)

            out = {
                "input_ids_stu": s_enc["input_ids"],
                "attention_mask_stu": s_enc["attention_mask"],
                "special_tokens_mask_stu": s_enc["special_tokens_mask"],
                "input_ids_tea": t_enc["input_ids"],
                "attention_mask_tea": t_enc["attention_mask"],
                "special_tokens_mask_tea": t_enc["special_tokens_mask"],
                "labels": torch.tensor(labels, dtype=torch.long),
            }
            if "token_type_ids" in s_enc:
                out["token_type_ids_stu"] = s_enc["token_type_ids"]
            if "token_type_ids" in t_enc:
                out["token_type_ids_tea"] = t_enc["token_type_ids"]

            # Offsets (fast tokenizers only)
            if "offset_mapping" in s_enc:
                out["offset_mapping_stu"] = s_enc["offset_mapping"]
            if "offset_mapping" in t_enc:
                out["offset_mapping_tea"] = t_enc["offset_mapping"]
            return out

        # ------- pair (bi-encoder) -------
        s1s, s2s = zip(*batch)

        s1_enc = self._encode(self.ts, s1s, self._stu_fast)
        s2_enc = self._encode(self.ts, s2s, self._stu_fast)

        t1_enc = self._encode(self.tt, s1s, self._tea_fast)
        t2_enc = self._encode(self.tt, s2s, self._tea_fast)

        out = {
            # student
            "input_ids1_stu": s1_enc["input_ids"],
            "attention_mask1_stu": s1_enc["attention_mask"],
            "special_tokens_mask1_stu": s1_enc["special_tokens_mask"],
            "input_ids2_stu": s2_enc["input_ids"],
            "attention_mask2_stu": s2_enc["attention_mask"],
            "special_tokens_mask2_stu": s2_enc["special_tokens_mask"],

            # teacher
            "input_ids1_tea": t1_enc["input_ids"],
            "attention_mask1_tea": t1_enc["attention_mask"],
            "special_tokens_mask1_tea": t1_enc["special_tokens_mask"],
            "input_ids2_tea": t2_enc["input_ids"],
            "attention_mask2_tea": t2_enc["attention_mask"],
            "special_tokens_mask2_tea": t2_enc["special_tokens_mask"],
        }

        if "token_type_ids" in s1_enc:
            out["token_type_ids1_stu"] = s1_enc["token_type_ids"]
        if "token_type_ids" in s2_enc:
            out["token_type_ids2_stu"] = s2_enc["token_type_ids"]
        if "token_type_ids" in t1_enc:
            out["token_type_ids1_tea"] = t1_enc["token_type_ids"]
        if "token_type_ids" in t2_enc:
            out["token_type_ids2_tea"] = t2_enc["token_type_ids"]

        # Offsets (fast tokenizers only)
        if "offset_mapping" in s1_enc:
            out["offset_mapping1_stu"] = s1_enc["offset_mapping"]
        if "offset_mapping" in s2_enc:
            out["offset_mapping2_stu"] = s2_enc["offset_mapping"]
        if "offset_mapping" in t1_enc:
            out["offset_mapping1_tea"] = t1_enc["offset_mapping"]
        if "offset_mapping" in t2_enc:
            out["offset_mapping2_tea"] = t2_enc["offset_mapping"]

        return out


print("Done Prepare Dataset")


In [ ]:
def info_nce(q, k, temperature=0.07, neg_valid_mask=None):
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)

    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss_inbatch = F.cross_entropy(logits, labels) 
    return loss_inbatch, logits

In [ ]:
data_train = '/kaggle/input/multitask-data/merged_9_data_3k_each_ver2.csv'
df = pd.read_csv(data_train)
df.head(3)

In [ ]:
df["premise"] = df["text"]
df["hypothesis"] = df["text"]
cols = ["premise", "hypothesis"]
df_out = df[cols].copy()

In [ ]:
student_name = "huawei-noah/TinyBERT_General_4L_312D"
tok_student = AutoTokenizer.from_pretrained(student_name)
model_student = AutoModel.from_pretrained(
    student_name,
    output_hidden_states=True
)

tok_teacher = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_teacher = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)

In [ ]:

train_ds = TextPairRaw(df_out, TASK_TYPE)

collate = DualTokenizerCollate(tok_student, tok_teacher, TASK_TYPE, MAX_LENGTH)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate, pin_memory=True, num_workers=2, persistent_workers=True)
print("Done Prepare Dataloader")

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from scipy.stats import pearsonr, spearmanr

# tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
tokenizer = tok_student 
class STSDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['score'], dtype=torch.float),
        }
        
def collate_fn(batch, tokenizer, max_len=128):
    s1_list = [item["sentence1"] for item in batch]
    s2_list = [item["sentence2"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )
    enc2 = tokenizer(
        s2_list,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "input_ids2": enc2["input_ids"],
        "attention_mask2": enc2["attention_mask"],
        "labels": labels,
    }

def eval_sts(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                # emb1 = mean_pooling(out1, attn1)
                # emb2 = mean_pooling(out2, attn2)
                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) * 2.5  # scale [-1,1] -> [0,5]
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    spearman_corr, _ = spearmanr(preds, labels)
    print(f"Spearman: {spearman_corr:.4f}")

    return spearman_corr


def eval_sts_task(model, path_list):
    model.eval()
    print(' ✅ eval_sts_task')
    for path in path_list:
        print(path)
        eval_dataset = STSDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_sts(model, eval_loader)
    model.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import datasets
import numpy as np
import torch

def eval_cls(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                label = batch["labels"]

                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                emb1 = out1.last_hidden_state[:, 0, :]
        
                preds.extend(emb1.cpu().numpy())
                labels.extend(label.numpy())
    
    return preds, labels

class ClasssifyDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return {
            "text": self.dataset.iloc[idx]['text'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.long),
        }

def clf_collate_fn(batch, tokenizer, max_len=512):
    s1_list = [item["text"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "labels": labels,
    }


def eval_classification_task(model, path_list):
    model.eval()
    print(' ✅ eval classifier')

    for train_path, dev_path in path_list:
        print(dev_path)
        eval_dataset = ClasssifyDataset(dev_path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )
        
        train_dataset = ClasssifyDataset(train_path)
        train_loader = DataLoader(
            train_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )

        X_train, y_train = eval_cls(model, train_loader)
        X_test, y_test = eval_cls(model, eval_loader)

        clf = LogisticRegression(
            random_state=42,
            n_jobs=1,
            max_iter=200,
            verbose=0,
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        scores = {}
        accuracy = accuracy_score(y_test, y_pred)
        scores["accuracy"] = accuracy
        f1 = f1_score(y_test, y_pred, average="macro")
        scores["f1"] = f1
        print(scores)
        
    model.train()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score


# tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')
tokenizer = tok_student 
class PairDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.float),
        }
        

def eval_pair(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) / 2
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    metric = get_metric_pair_classification(preds, labels)
    print(metric)

    return metric

def get_metric_pair_classification(scores, labels):
    best_acc, best_thr = 0, 0
    for thr in np.linspace(0, 1, 200):
        preds = (scores >= thr).astype(int)
        acc = accuracy_score(labels, preds)
        if acc > best_acc:
            best_acc, best_thr = acc, thr
    preds = (scores >= best_thr).astype(int)
    return {
        "best_threshold": best_thr,
        "accuracy": best_acc,
        "f1": f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "average_precision": average_precision_score(labels, scores)
    }


def eval_pair_task(model, path_list):
    model.eval()
    print(' ✅ eval_pair_task')
    for path in path_list:
        print(path)
        eval_dataset = PairDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_pair(model, eval_loader)
    model.train()

In [ ]:
import torch
import torch.nn as nn
import math
def find_best_mapping(x, base_tokens, blending_special, base_special, best_one=True):
    tmp_x = x.replace(blending_special, base_special)
    if tmp_x in base_tokens:
        return tmp_x, tmp_x
    else:
        if best_one:
            best = None
            best_dist = None
            for y in base_tokens:
                d = editdistance.eval(tmp_x, y)
                if best is None or d < best_dist:
                    best = y
                    best_dist = d
            return tmp_x, best
        else:
            token_and_distance = [(y, editdistance.eval(tmp_x, y)) for y in base_tokens]
            min_distance = min(d for _, d in token_and_distance)
            shortest_distance_tokens = [y for y, d in token_and_distance if d == min_distance]
            return tmp_x, shortest_distance_tokens
class CKALoss(nn.Module):
    def __init__(self, eps ):
        super().__init__()
        self.eps = eps
    def forward(self, SH, TH): 
        dT = TH.size(-1)
        dS = SH.size(-1)
        SH = SH.view(-1,dS).to(SH.device,torch.float64)
        TH = TH.view(-1,dT).to(SH.device,torch.float64)
        
        slen = SH.size(0)
        SH = SH - SH.mean(0, keepdim=True)
        TH = TH - TH.mean(0, keepdim=True)
                
        num = torch.norm(SH.t().matmul(TH),'fro')
        den1 = torch.norm(SH.t().matmul(SH),'fro') + self.eps
        den2 = torch.norm(TH.t().matmul(TH),'fro') + self.eps
        
        return 1 - num/torch.sqrt(den1*den2)
def build_reciprocal_mapping_from_token_lists(
    teacher_tokens, student_tokens,
    teacher_special="<s>", student_special="[CLS]"
):
    teacher_to_student = align_tokens(teacher_tokens, student_tokens, teacher_special, student_special)
    student_to_teacher = align_tokens(student_tokens, teacher_tokens, student_special, teacher_special)

    reciprocal_mapping = {}
    for t, s in teacher_to_student.items():
        if s in student_to_teacher and student_to_teacher[s] == t:
            reciprocal_mapping[t] = s
    return reciprocal_mapping

def filter_mapping_top_k(teacher_tokens, importance_vec, reciprocal_mapping, k):
    if len(reciprocal_mapping) == 0:
        return {}

    token_scores = {}
    for idx, tok in enumerate(teacher_tokens):
        if tok in reciprocal_mapping:
            score = importance_vec[idx].item()
            if tok not in token_scores or score > token_scores[tok]:
                token_scores[tok] = score

    if len(token_scores) == 0:
        return {}

    k = max(1, min(k, len(token_scores)))
    top_tokens = sorted(token_scores.items(), key=lambda x: x[1], reverse=True)[:k]
    allowed = {tok for tok, _ in top_tokens}

    filtered = {t: s for t, s in reciprocal_mapping.items() if t in allowed}
    return filtered

def get_indices_from_mapping_tokens(teacher_tokens, student_tokens, reciprocal_mapping):
    if len(reciprocal_mapping) == 0:
        return [], []

    teacher_indices = [idx for idx, tok in enumerate(teacher_tokens) if tok in reciprocal_mapping]

    mapped_student_tokens = set(reciprocal_mapping.values())
    student_indices = [idx for idx, tok in enumerate(student_tokens) if tok in mapped_student_tokens]

    return teacher_indices, student_indices

def align_indices_by_mapping(teacher_indices, student_indices, reciprocal_top, teacher_tokens, student_tokens):
    aligned_teacher = []
    aligned_student = []

    s2t = {v:k for k,v in reciprocal_top.items()}

    for t_idx in teacher_indices:
        t_tok = teacher_tokens[t_idx]
        if t_tok not in reciprocal_top:
            continue
        s_tok = reciprocal_top[t_tok]
        for idx_s, tok_s in enumerate(student_tokens):
            if tok_s == s_tok:
                aligned_teacher.append(t_idx)
                aligned_student.append(idx_s)
                break

    return aligned_teacher, aligned_student




In [ ]:
# --- Edit distance backend ---
try:
    import Levenshtein as _Lev
    def _edit_dist(a: str, b: str) -> int:
        return _Lev.distance(a, b)
except Exception:
    def _edit_dist(a: str, b: str) -> int:
        # pure-python Levenshtein distance
        if a == b:
            return 0
        if len(a) == 0:
            return len(b)
        if len(b) == 0:
            return len(a)
        if len(a) < len(b):
            a, b = b, a
        prev = list(range(len(b) + 1))
        for i, ca in enumerate(a, start=1):
            cur = [i]
            for j, cb in enumerate(b, start=1):
                ins = cur[j-1] + 1
                dele = prev[j] + 1
                sub = prev[j-1] + (ca != cb)
                cur.append(min(ins, dele, sub))
            prev = cur
        return prev[-1]

def _clean_tok(t: str) -> str:
    if t is None:
        return ""
    # SentencePiece / GPT / WordPiece cleanup
    t = t.replace("▁", "").replace("Ġ", "")
    if t.startswith("##"):
        t = t[2:]
    return t.strip().lower()

def compute_token_importance(attention_weights: torch.Tensor) -> torch.Tensor:
    '''
    attention_weights: [H, L, L] or [L, L]
    Importance = column-sum of avg attention -> softmax -> probs over tokens.
    '''
    if attention_weights.dim() == 3:
        avg_att = attention_weights.mean(dim=0)   # [L, L]
    else:
        avg_att = attention_weights               # [L, L]
    scores = avg_att.sum(dim=0)                   # [L]
    return torch.softmax(scores, dim=-1)

def align_tokens(teacher_tokens, student_tokens, teacher_special="<s>", student_special="[CLS]"):
    '''
    Return a dict mapping teacher_token_string -> student_token_string (best match).
    Used by build_reciprocal_mapping_from_token_lists() in compute_att_loss_2.
    '''
    teacher_to_student = {}
    if (not teacher_tokens) or (not student_tokens):
        return teacher_to_student

    # Map special token if both exist
    if teacher_special in teacher_tokens and student_special in student_tokens:
        teacher_to_student[teacher_special] = student_special

    student_set = set(student_tokens)

    # Pre-clean student tokens once
    stu_clean = [(s, _clean_tok(s)) for s in student_tokens if s != student_special]

    for t in teacher_tokens:
        if t == teacher_special:
            continue

        # quick exact match after special replacement
        tmp_t = t.replace(teacher_special, student_special)
        if tmp_t in student_set and tmp_t != student_special:
            teacher_to_student[t] = tmp_t
            continue

        t_clean = _clean_tok(tmp_t)
        if t_clean == "":
            continue

        best_s, best_d = None, 10**9
        for s, s_clean in stu_clean:
            if t_clean == s_clean:
                best_s, best_d = s, 0
                break
            d = _edit_dist(t_clean, s_clean)
            if d < best_d:
                best_s, best_d = s, d

        if best_s is not None:
            teacher_to_student[t] = best_s

    return teacher_to_student

# --- Matryoshka (nested-prefix) cosine distillation loss ---
def matryoshka_prefix_cosine_loss(student_emb: torch.Tensor,
                                  teacher_emb: torch.Tensor,
                                  dims=(128, 256, 512),
                                  eps: float = 1e-8) -> torch.Tensor:
    '''
    student_emb, teacher_emb: [B, D] (same D)
    Loss = average over d in dims plus full D of (1 - cosine(student[:d], teacher[:d])).
    '''
    assert student_emb.dim() == 2 and teacher_emb.dim() == 2
    B, D = student_emb.shape
    assert teacher_emb.shape == (B, D)

    dims_list = [int(d) for d in dims if (d is not None and int(d) > 0 and int(d) <= D)]
    if D not in dims_list:
        dims_list.append(D)
    dims_list = sorted(set(dims_list))

    total = 0.0
    for d in dims_list:
        s = student_emb[:, :d].float()
        t = teacher_emb[:, :d].float()
        s = s / (s.norm(dim=-1, keepdim=True) + eps)
        t = t / (t.norm(dim=-1, keepdim=True) + eps)
        total = total + (1.0 - (s * t).sum(dim=-1)).mean()
    return total / len(dims_list)

def extract_teacher_sentence_embedding(T_last: torch.Tensor,
                                      input_ids: torch.Tensor,
                                      attention_mask: torch.Tensor,
                                      tok_teacher,
                                      embed_token: str = None):
    '''
    Returns teacher sentence embedding [B, d_t].
    Priority:
      1) If embed_token is provided and present in the sequence: use its hidden state.
      2) Else: mean pooling.
    This supports LLM2Vec-style embedding tokens if they exist.
    '''
    if embed_token is not None:
        try:
            embed_id = tok_teacher.convert_tokens_to_ids(embed_token)
            # Guard: only use if the token is really in vocab (avoid unk collisions)
            try:
                if embed_token not in tok_teacher.get_vocab():
                    embed_id = None
            except Exception:
                pass
        except Exception:
            embed_id = None

        if embed_id is not None:
            m = (input_ids == embed_id)  # [B, L]
            if m.any():
                idx = m.float().argmax(dim=1)  # [B]
                has_any = m.any(dim=1)         # [B]
                out = T_last[:, 0, :].clone()
                b_idx = torch.arange(T_last.size(0), device=T_last.device)
                out[has_any] = T_last[b_idx[has_any], idx[has_any], :]
                return out

    # Fallback: mean pooling
    mask = attention_mask.unsqueeze(-1).type_as(T_last)
    summed = (T_last * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts


In [ ]:
eval_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_validation.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_validation.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_validation.csv')]

eval_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_validation.csv']

eval_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_validation.csv']
test_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_test.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_test.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_test.csv')]

test_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_test.csv']

test_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_test.csv']

# model_student.to("cuda")
# eval_classification_task(model_student, test_cls_tasks)
# eval_pair_task(model_student, test_pair_tasks)
# eval_sts_task(model_student, test_sts_tasks)

In [ ]:
from torch import Tensor
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


In [ ]:
best_f1_macro = -1.0
all_preds, all_labels = [], []
total_loss = 0.0

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_steps = len(train_loader)
total_traning_steps = num_steps * epochs
warmup_ratio = 0.1

def pick_devices():
    if torch.cuda.device_count() >= 2:
        dev_s = torch.device("cuda:0")  # student
        dev_t = torch.device("cuda:1")  # teacher
    elif torch.cuda.is_available():
        print("[WARN] Only 1 GPU available -> both on cuda:0")
        dev_s = dev_t = torch.device("cuda:0")
    else:
        print("[WARN] No GPU -> CPU")
        dev_s = dev_t = torch.device("cpu")
    return dev_s, dev_t

device_s, device_t = pick_devices()

model_student.to(device_s)
model_teacher.to(device_t)
model_teacher.eval()
for p in model_teacher.parameters():
    p.requires_grad_(False)

proj_s2t = None
proj_t2s = None

optimizer = optim.AdamW(model_student.parameters(), lr=LR)
scaler = GradScaler(enabled=torch.cuda.is_available())
scheduler = get_scheduler(
    name='cosine_with_min_lr',
    optimizer=optimizer,
    num_warmup_steps=int(total_traning_steps * warmup_ratio),
    # num_warmup_steps=1,
    num_training_steps=total_traning_steps,
    scheduler_specific_kwargs={'min_lr': 2e-6}
)
if save_dir:
    os.makedirs(save_dir, exist_ok=True)

n_items = 0
use_task_loss = True

In [ ]:
# TUNING KNOBS
# Sentence embedding pooling for student (SimCSE + MRL use this)
# "cls" = token at position 0; "mean" = masked mean pooling
student_pool = "mean"   # {"cls","mean"}

# Overall weights
w_task   = 0.5     # SimCSE InfoNCE weight
alpha_kd = 0.5     # total KD weight (applies to alpha_attn*att_loss + beta_mrl*mrl_loss)

# ---- MRL (Matryoshka) ----
beta_mrl_max   = 1.0      # final MRL weight inside KD
mrl_start      = 0        # start ramp at this global_step
mrl_ramp       = 1000     # steps to ramp from 0 -> beta_mrl_max
mrl_weight_mode = "inverse_sqrt"  # {"uniform","inverse_sqrt","inverse","log"}

# Matryoshka prefix dimensions (filtered by hidden size automatically)
# d_s is defined later after models are loaded; we set dims after d_s is known.

# ---- CKA / IRA (attention alignment) ----
alpha_attn_max = 0.02     # final attention weight inside KD (keep small)
att_start      = 1500     # start ramp for attention loss
att_ramp       = 2500     # steps to ramp from 0 -> alpha_attn_max
att_every      = 4        # compute attention loss every N steps (reduce overhead)
top_frac       = 0.25     # fraction of mapped tokens to align (0.2~0.33 stable)
min_tokens     = 2        # minimum aligned tokens required to compute CKA
k_ira          = 1        # last-k layers for IRA/CKA

def ramp(step: int, start: int, ramp_steps: int) -> float:
    if step < start:
        return 0.0
    if ramp_steps <= 0:
        return 1.0
    return min(1.0, (step - start) / float(ramp_steps))

In [ ]:
# KD UTILITIES: token alignment, CKA, MRL, teacher embedding

# pooling
def mean_pooling(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

def get_student_sentence_emb(last_hidden: torch.Tensor, att_mask: torch.Tensor, mode: str) -> torch.Tensor:
    if mode == "mean":
        return mean_pooling(last_hidden, att_mask)
    return last_hidden[:, 0, :]  # "cls"

# robust edit distance (optional)
try:
    import Levenshtein as _Lev
    def _edit_dist(a: str, b: str) -> int:
        return _Lev.distance(a, b)
except Exception:
    def _edit_dist(a: str, b: str) -> int:
        # pure python levenshtein
        if a == b:
            return 0
        if len(a) == 0:
            return len(b)
        if len(b) == 0:
            return len(a)
        if len(a) < len(b):
            a, b = b, a
        prev = list(range(len(b) + 1))
        for i, ca in enumerate(a, start=1):
            cur = [i]
            for j, cb in enumerate(b, start=1):
                ins = cur[j-1] + 1
                dele = prev[j] + 1
                sub = prev[j-1] + (ca != cb)
                cur.append(min(ins, dele, sub))
            prev = cur
        return prev[-1]

def _clean_tok(t: str) -> str:
    if t is None:
        return ""
    t = t.replace("▁", "").replace("Ġ", "")
    if t.startswith("##"):
        t = t[2:]
    return t.strip().lower()

# token importance (FIXED SIGNATURE)
def compute_token_importance(attention_weights: torch.Tensor, tokens=None) -> torch.Tensor:
    '''
    attention_weights: [H, L, L] or [L, L]
    tokens: optional list[str] of length L (may include special tokens)
    Returns: probs [L]
    '''
    if attention_weights.dim() == 3:
        avg_att = attention_weights.mean(dim=0)   # [L, L]
    else:
        avg_att = attention_weights               # [L, L]
    scores = avg_att.sum(dim=0)                   # [L]

    # Downweight obvious special tokens if tokens are provided
    if tokens is not None and isinstance(tokens, (list, tuple)) and len(tokens) == scores.numel():
        special = {
            "[PAD]", "<pad>", "</s>", "<s>", "[CLS]", "[SEP]", "[UNK]",
            "<|pad|>", "<|endoftext|>", "<|eos|>", "<|bos|>", "<|embed|>"
        }
        scores = scores.clone()
        bad = [i for i,t in enumerate(tokens) if t in special]
        if 0 < len(bad) < scores.numel():
            scores[bad] = -1e9

    return torch.softmax(scores, dim=-1)

# token alignment
def align_tokens(teacher_tokens, student_tokens, teacher_special="<s>", student_special="[CLS]"):
    '''
    Map each teacher token string -> closest student token string.
    Used by IRA/CKA when teacher/student tokenizers differ.
    '''
    out = {}
    if (not teacher_tokens) or (not student_tokens):
        return out

    student_set = set(student_tokens)

    # special token mapping
    if teacher_special in teacher_tokens and student_special in student_tokens:
        out[teacher_special] = student_special

    stu_clean = [(s, _clean_tok(s)) for s in student_tokens if s != student_special]

    for t in teacher_tokens:
        if t == teacher_special:
            continue
        tmp_t = t.replace(teacher_special, student_special)
        if tmp_t in student_set and tmp_t != student_special:
            out[t] = tmp_t
            continue

        t_clean = _clean_tok(tmp_t)
        if t_clean == "":
            continue

        best_s, best_d = None, 10**9
        for s, s_clean in stu_clean:
            if t_clean == s_clean:
                best_s, best_d = s, 0
                break
            d = _edit_dist(t_clean, s_clean)
            if d < best_d:
                best_s, best_d = s, d
        if best_s is not None:
            out[t] = best_s

    return out

def build_reciprocal_mapping_from_token_lists(teacher_tokens, student_tokens,
                                              teacher_special="<s>", student_special="[CLS]"):
    teacher_to_student = align_tokens(teacher_tokens, student_tokens, teacher_special, student_special)
    student_to_teacher = align_tokens(student_tokens, teacher_tokens, student_special, teacher_special)

    reciprocal = {}
    for t, s in teacher_to_student.items():
        if s in student_to_teacher and student_to_teacher[s] == t:
            reciprocal[t] = s
    return reciprocal

# Linear CKA (stable)
class LinearCKALoss(nn.Module):
    '''
    Linear CKA: ||X^T Y||_F^2 / (||X^T X||_F * ||Y^T Y||_F)
    Loss = 1 - CKA in [0,1] (approximately).
    '''
    def __init__(self, eps: float = 1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, X: torch.Tensor, Y: torch.Tensor) -> torch.Tensor:
        # X,Y: [N, D]
        X = X.float()
        Y = Y.float()
        X = X - X.mean(dim=0, keepdim=True)
        Y = Y - Y.mean(dim=0, keepdim=True)

        XT_Y = X.t().matmul(Y)
        XT_X = X.t().matmul(X)
        YT_Y = Y.t().matmul(Y)

        num = (XT_Y ** 2).sum()  # ||X^T Y||_F^2
        den = torch.norm(XT_X, p="fro") * torch.norm(YT_Y, p="fro")
        return 1.0 - (num / (den + self.eps))

# IRA/CKA attention alignment
def _build_span_overlap_matrix(offsets_tea, offsets_stu, L_t: int, L_s: int, device, eps: float = 1e-12):
    if offsets_tea is None or offsets_stu is None:
        return None, 0.0, 0.0

    # Ensure CPU lists for fast monotonic scan
    if torch.is_tensor(offsets_tea):
        ot = offsets_tea[:L_t].detach().cpu().tolist()
    else:
        ot = list(offsets_tea)[:L_t]
    if torch.is_tensor(offsets_stu):
        os_ = offsets_stu[:L_s].detach().cpu().tolist()
    else:
        os_ = list(offsets_stu)[:L_s]

    A = torch.zeros((L_s, L_t), device=device, dtype=torch.float32)

    i = 0  # teacher
    j = 0  # student
    # Monotonic two-pointer overlap fill
    while i < L_t and j < L_s:
        a, b = ot[i]
        c, d = os_[j]

        # skip zero-length spans (special tokens)
        if b <= a:
            i += 1
            continue
        if d <= c:
            j += 1
            continue

        if b <= c:
            i += 1
            continue
        if d <= a:
            j += 1
            continue

        overlap = min(b, d) - max(a, c)
        if overlap > 0:
            A[j, i] = float(overlap)

        # advance whichever span ends first
        if b < d:
            i += 1
        elif d < b:
            j += 1
        else:
            i += 1
            j += 1

    row_sum = A.sum(dim=1, keepdim=True)
    A = A / row_sum.clamp_min(eps)

    coverage_s = float((row_sum.squeeze(1) > 0).float().mean().item())
    coverage_t = float((A.sum(dim=0) > 0).float().mean().item())
    return A, coverage_s, coverage_t


def _project_teacher_attention_to_student(Attn_t: torch.Tensor, A: torch.Tensor) -> torch.Tensor:
    """Project teacher attention into student token space using A.

    Attn_t: [H, N, N]
    A:      [M, N] (row-normalized)
    Returns:
        Attn_t_proj: [H, M, M] where Attn_t_proj = A @ Attn_t @ A^T
    """
    # left: (M,N) x (H,N,N) -> (H,M,N)
    tmp = torch.einsum("mn,hnk->hmk", A, Attn_t)
    # right: (H,M,N) x (N,M) -> (H,M,M)
    return torch.einsum("hmn,nk->hmk", tmp, A.transpose(0, 1))

# Gram centering utilities
def _center_gram(K: torch.Tensor, unbiased: bool = False) -> torch.Tensor:
    """
    Center a Gram matrix K in RKHS sense.
    K: [N, N]
    """
    if not unbiased:
        mean_row = K.mean(dim=0, keepdim=True)
        mean_col = K.mean(dim=1, keepdim=True)
        mean_all = K.mean()
        return K - mean_row - mean_col + mean_all

    # Unbiased-ish centering for small N (often reduces bias).
    # Works well when N is not huge.
    N = K.size(0)
    if N <= 2:
        return K - K.mean()

    K = K.clone()
    K.fill_diagonal_(0)
    means = K.sum(dim=0) / (N - 2)
    means = means - means.sum() / (2 * (N - 1))
    K = K - means.unsqueeze(0) - means.unsqueeze(1)
    K.fill_diagonal_(0)
    return K


# Attention transforms
def _remove_diag_and_renorm(att: torch.Tensor, eps: float) -> torch.Tensor:
    """
    att: [H, N, N] or [H, N, D] (diagonal removal only if last dim == N)
    """
    H, N, D = att.shape
    if D != N:
        return att

    eye = torch.eye(N, device=att.device, dtype=att.dtype).unsqueeze(0)  # [1,N,N]
    att = att * (1.0 - eye)  # zero diagonal
    att = att / (att.sum(dim=-1, keepdim=True).clamp_min(eps))
    return att


def _attn_transform(
    att: torch.Tensor,
    transform: str = "clr",     # "none" | "sqrt" | "log" | "clr"
    eps: float = 1e-8,
) -> torch.Tensor:
    """
    Transform attention distributions to make them more comparable for CKA.

    att: [H, N, D], rows are distributions over keys (D).
    """
    att = att.clamp_min(eps)

    if transform == "none":
        return att
    if transform == "sqrt":
        return torch.sqrt(att)
    if transform == "log":
        return torch.log(att)
    if transform == "clr":
        # centered log-ratio: log(p) - mean(log(p)) per row
        logp = torch.log(att)
        return logp - logp.mean(dim=-1, keepdim=True)

    raise ValueError(f"Unknown transform={transform}")


def _row_entropy(att: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Compute entropy per query row from attention distribution.
    att: [N, D] (already row-normalized probs)
    returns: [N]
    """
    p = att.clamp_min(eps)
    return -(p * torch.log(p)).sum(dim=-1)


def _select_queries_by_focus(
    att_mean: torch.Tensor,        # [N, N] mean over heads for a square attn map
    max_q: int,
    eps: float = 1e-8
) -> torch.Tensor:
    """
    Select query indices using focus score = 1 - normalized_entropy.
    If max_q >= N => all.
    """
    N = att_mean.size(0)
    if max_q is None or max_q >= N:
        return torch.arange(N, device=att_mean.device)

    # Ensure row-normalized
    A = att_mean.clamp_min(eps)
    A = A / A.sum(dim=-1, keepdim=True).clamp_min(eps)

    ent = _row_entropy(A, eps=eps)  # [N]
    ent_norm = ent / math.log(float(N) + 1e-12)  # in [0,1] roughly
    focus = 1.0 - ent_norm  # higher = sharper/more informative

    top = torch.topk(focus, k=max_q, largest=True).indices
    return torch.sort(top).values


# Multi-head CKA
class MultiHeadCKALoss(nn.Module):
    """
    CKA computed in Gram space using multi-head kernel aggregation:

        K = (1/H) * sum_h (X_h X_h^T)
        L = (1/H')* sum_h (Y_h Y_h^T)

    Then:
        CKA(K,L) = <Kc, Lc>_F / (||Kc||_F ||Lc||_F)
        loss = 1 - CKA

    """
    def __init__(
        self,
        eps: float = 1e-8,
        unbiased_center: bool = False,
        transform: str = "clr",     # good default for attention probs
        remove_diag: bool = True,
        use_fp64: bool = True,
    ):
        super().__init__()
        self.eps = eps
        self.unbiased_center = unbiased_center
        self.transform = transform
        self.remove_diag = remove_diag
        self.use_fp64 = use_fp64

    def _multihead_linear_gram(self, att: torch.Tensor) -> torch.Tensor:
        """
        att: [H, N, D] (D typically == N after token selection)
        returns: [N, N] Gram over queries
        """
        if self.remove_diag:
            att = _remove_diag_and_renorm(att, eps=self.eps)

        # Row-normalize (important if attentions are not perfectly normalized after slicing)
        att = att.clamp_min(self.eps)
        att = att / att.sum(dim=-1, keepdim=True).clamp_min(self.eps)

        # Transform in probability space (CLR/log/sqrt)
        att = _attn_transform(att, transform=self.transform, eps=self.eps)

        # Use fp64 for stability if desired
        if self.use_fp64:
            att = att.double()

        # Compute K[n,m] = (1/H) * sum_h sum_k att[h,n,k] * att[h,m,k]
        # einsum does: sum over h,k
        K = torch.einsum("hnk,hmk->nm", att, att)
        K = K / float(att.size(0))  # average over heads

        # Back to fp32 for downstream
        return K.float() if self.use_fp64 else K

    def forward(self, att_s: torch.Tensor, att_t: torch.Tensor) -> torch.Tensor:
        """
        att_s: [Hs, N, D]
        att_t: [Ht, N, D]
        """
        N = att_s.size(1)
        if N <= 1:
            return torch.zeros((), device=att_s.device)

        K = self._multihead_linear_gram(att_s)  # [N,N]
        L = self._multihead_linear_gram(att_t)  # [N,N]

        Kc = _center_gram(K, unbiased=self.unbiased_center)
        Lc = _center_gram(L, unbiased=self.unbiased_center)

        num = (Kc * Lc).sum()
        den = torch.sqrt((Kc * Kc).sum() * (Lc * Lc).sum()).clamp_min(self.eps)

        cka = (num / den).clamp(min=-1.0, max=1.0)
        return 1.0 - cka


def compute_att_loss_2(
    teacher_atts, student_atts,
    input_ids_tea, input_ids_stu,
    attention_mask_tea, attention_mask_stu,
    tok_teacher, tok_student,
    k: int,
    device,
    teacher_special="<s>",
    student_special="[CLS]",
    top_frac: float = 0.25,
    min_tokens: int = 2,
    offset_mapping_tea=None,
    offset_mapping_stu=None,
    min_coverage: float = 0.30,
    cka_transform: str = "clr",          # "clr" tends to beat plain linear on attention probs
    cka_unbiased_center: bool = False,
    cka_remove_diag: bool = True,
    cka_use_fp64: bool = True,
    query_sample: int = None,            # e.g. 96 or 128; None = use all
):
    """
    IRA/CKA attention alignment with Span-MinED (preferred) + MinED fallback.
    This version uses MultiHeadCKA in Gram space (significantly stronger than
    head-averaged LinearCKA with feature truncation).

    Expected attn shapes per layer:
      teacher_atts[layer] : [B, Ht, Lt, Lt]
      student_atts[layer] : [B, Hs, Ls, Ls]
    """
    cka = MultiHeadCKALoss(
        eps=1e-8,
        unbiased_center=cka_unbiased_center,
        transform=cka_transform,
        remove_diag=cka_remove_diag,
        use_fp64=cka_use_fp64,
    ).to(device)

    # Use last-k layers
    teacher_last_k = teacher_atts[-k:]
    student_last_k = student_atts[-k:]

    B = input_ids_tea.size(0)
    att_loss_total = torch.zeros((), device=device)
    n_terms = 0

    for b in range(B):
        Lt = int(attention_mask_tea[b].sum().item())
        Ls = int(attention_mask_stu[b].sum().item())
        if Lt <= 1 or Ls <= 1:
            continue

        
        # Try Span-MinED soft alignment
        A_full, cov_s, cov_t = None, 0.0, 0.0
        if offset_mapping_tea is not None and offset_mapping_stu is not None:
            try:
                A_full, cov_s, cov_t = _build_span_overlap_matrix(
                    offset_mapping_tea[b], offset_mapping_stu[b],
                    L_t=Lt, L_s=Ls, device=device
                )
            except Exception:
                A_full = None

        use_span = (A_full is not None) and (cov_s >= min_coverage) and (cov_t >= min_coverage)

        if use_span:
            # token list for special-token downweighting inside compute_token_importance
            tea_tokens = tok_teacher.convert_ids_to_tokens(input_ids_tea[b, :Lt].detach().cpu().tolist())

            # ---- teacher token importance from LAST teacher layer ----
            last_teacher_att = teacher_last_k[-1][b, :, :Lt, :Lt]  # [Ht, Lt, Lt]
            tea_imp = compute_token_importance(last_teacher_att, tea_tokens)  # [Lt]

            # Only teacher tokens that overlap any student token
            valid_cols = (A_full.sum(dim=0) > 0)  # [Lt]
            if valid_cols.sum().item() < min_tokens:
                use_span = False
            else:
                imp = tea_imp.clone()
                imp[~valid_cols] = -1e9

                k_top = max(min_tokens, int(math.ceil(top_frac * float(valid_cols.sum().item()))))
                idx_t = torch.topk(imp, k=k_top, largest=True).indices
                idx_t = torch.sort(idx_t).values  # keep order

                # Student rows that receive any mass after restricting teacher columns
                A_sub = A_full[:, idx_t]                    # [Ls, N]
                row_sum = A_sub.sum(dim=1)                  # [Ls]
                idx_s = torch.nonzero(row_sum > 1e-12).squeeze(-1)

                if idx_s.numel() < min_tokens:
                    use_span = False
                else:
                    # Normalize rows on the restricted A
                    A_use = A_sub[idx_s]  # [M, N]
                    A_use = A_use / A_use.sum(dim=1, keepdim=True).clamp_min(1e-12)

                    # confidence gate based on span coverage
                    conf = float(min(cov_s, cov_t))

                    # If sampling queries, decide query subset using teacher projected attention
                    # (use last layer for scoring, then reuse subset for all compared layers)
                    q_idx = None
                    if query_sample is not None:
                        tea_last = teacher_last_k[-1][b, :, :Lt, :Lt]       # [Ht, Lt, Lt]
                        tea_sel = tea_last[:, idx_t][:, :, idx_t]           # [Ht, N, N]
                        tea_proj_last = _project_teacher_attention_to_student(tea_sel, A_use)  # [Ht, M, M]
                        tea_mean = tea_proj_last.mean(dim=0)                # [M, M]
                        q_idx = _select_queries_by_focus(tea_mean, max_q=query_sample)
                    # else q_idx = all

                    for tea_layer, stu_layer in zip(teacher_last_k, student_last_k):
                        tea = tea_layer[b, :, :Lt, :Lt]  # [Ht, Lt, Lt]
                        stu = stu_layer[b, :, :Ls, :Ls]  # [Hs, Ls, Ls]

                        # select teacher tokens (N) then project to student space (M)
                        tea_sel = tea[:, idx_t][:, :, idx_t]               # [Ht, N, N]
                        tea_proj = _project_teacher_attention_to_student(tea_sel, A_use)  # [Ht, M, M]

                        # select student token subset (M)
                        stu_sel = stu[:, idx_s][:, :, idx_s]               # [Hs, M, M]

                        # optionally sample queries (reduce N in Gram)
                        if q_idx is not None:
                            tea_proj = tea_proj[:, q_idx][:, :, q_idx]     # [Ht, Q, Q]
                            stu_sel = stu_sel[:, q_idx][:, :, q_idx]       # [Hs, Q, Q]

                        # For CKA, treat each query row as sample, keys as features:
                        # reshape to [H, N, D] where D == N
                        # Here: [H, Q, Q] already fits with N=Q, D=Q
                        att_loss_total = att_loss_total + conf * cka(stu_sel, tea_proj)
                        n_terms += 1

        
        #  Fallback: MinED-like hard token map
        if not use_span:
            tea_tokens = tok_teacher.convert_ids_to_tokens(input_ids_tea[b, :Lt].detach().cpu().tolist())
            stu_tokens = tok_student.convert_ids_to_tokens(input_ids_stu[b, :Ls].detach().cpu().tolist())

            # teacher token importance from last layer
            last_teacher_att = teacher_last_k[-1][b, :, :Lt, :Lt]  # [Ht, Lt, Lt]
            tea_imp = compute_token_importance(last_teacher_att, tea_tokens)  # [Lt]

            reciprocal = build_reciprocal_mapping_from_token_lists(
                tea_tokens, stu_tokens,
                teacher_special=teacher_special,
                student_special=student_special,
            )
            if len(reciprocal) == 0:
                continue

            # score reciprocal teacher tokens by importance
            tok_score = {}
            for t_idx, t_tok in enumerate(tea_tokens):
                if t_tok in reciprocal:
                    tok_score[t_tok] = max(tok_score.get(t_tok, -1e9), float(tea_imp[t_idx].item()))

            if len(tok_score) < min_tokens:
                continue

            k_top = max(min_tokens, int(math.ceil(top_frac * len(tok_score))))
            keep_teacher = [t for t, _ in sorted(tok_score.items(), key=lambda x: x[1], reverse=True)[:k_top]]

            # build aligned (t_idx, s_idx) pairs; sort by student order for consistency
            pairs = []
            for t_tok in keep_teacher:
                s_tok = reciprocal[t_tok]
                try:
                    t_idx = tea_tokens.index(t_tok)
                    s_idx = stu_tokens.index(s_tok)
                except ValueError:
                    continue
                if t_idx < Lt and s_idx < Ls:
                    pairs.append((t_idx, s_idx))

            if len(pairs) < min_tokens:
                continue

            pairs = sorted(pairs, key=lambda x: x[1])  # sort by student idx
            idx_t = torch.tensor([p[0] for p in pairs], device=device, dtype=torch.long)
            idx_s = torch.tensor([p[1] for p in pairs], device=device, dtype=torch.long)

            for tea_layer, stu_layer in zip(teacher_last_k, student_last_k):
                tea = tea_layer[b, :, :Lt, :Lt]  # [Ht, Lt, Lt]
                stu = stu_layer[b, :, :Ls, :Ls]  # [Hs, Ls, Ls]

                # Compare attention restricted to aligned token set (no min(d) truncation!)
                tea_sel = tea[:, idx_t][:, :, idx_t]  # [Ht, N, N]
                stu_sel = stu[:, idx_s][:, :, idx_s]  # [Hs, N, N]

                # optional query sampling
                if query_sample is not None and idx_s.numel() > query_sample:
                    tea_mean = tea_sel.mean(dim=0)  # [N,N]
                    q_idx = _select_queries_by_focus(tea_mean, max_q=query_sample)
                    tea_sel = tea_sel[:, q_idx][:, :, q_idx]
                    stu_sel = stu_sel[:, q_idx][:, :, q_idx]

                att_loss_total = att_loss_total + cka(stu_sel, tea_sel)
                n_terms += 1

    if n_terms == 0:
        return torch.zeros((), device=device)
    return att_loss_total / n_terms

# Matryoshka (MRL)
def matryoshka_prefix_cosine_loss(student_emb: torch.Tensor,
                                  teacher_emb: torch.Tensor,
                                  dims,
                                  weight_mode: str = "inverse_sqrt",
                                  eps: float = 1e-8) -> torch.Tensor:
    '''
    student_emb, teacher_emb: [B, D] with same D.
    For each prefix dimension d in dims (+ full D), compute 1 - cosine(student[:d], teacher[:d]).
    Optionally weight smaller dims more.
    '''
    assert student_emb.dim() == 2 and teacher_emb.dim() == 2
    B, D = student_emb.shape
    assert teacher_emb.shape == (B, D)

    dims_list = [int(d) for d in dims if d is not None and int(d) > 0 and int(d) <= D]
    if D not in dims_list:
        dims_list.append(D)
    dims_list = sorted(set(dims_list))

    def _w(d):
        if weight_mode == "uniform":
            return 1.0
        if weight_mode == "inverse":
            return 1.0 / max(1.0, float(d))
        if weight_mode == "log":
            return 1.0 / math.log(2.0 + float(d))
        # default: inverse_sqrt
        return 1.0 / math.sqrt(max(1.0, float(d)))

    total = 0.0
    wsum = 0.0
    for d in dims_list:
        s = student_emb[:, :d].float()
        t = teacher_emb[:, :d].float()
        s = s / (s.norm(dim=-1, keepdim=True) + eps)
        t = t / (t.norm(dim=-1, keepdim=True) + eps)
        term = (1.0 - (s * t).sum(dim=-1)).mean()
        w = _w(d)
        total += w * term
        wsum += w
    return total / max(wsum, 1e-9)

# teacher sentence embedding
def extract_teacher_sentence_embedding(last_hidden_state: torch.Tensor,
                                      input_ids: torch.Tensor,
                                      attention_mask: torch.Tensor,
                                      tokenizer,
                                      embed_token: str = None) -> torch.Tensor:
    '''
    If embed_token is present in the sequence, use its hidden state.
    Otherwise use mean pooling.
    Returns: [B, d_t]
    '''
    if embed_token is not None:
        try:
            embed_id = tokenizer.convert_tokens_to_ids(embed_token)
        except Exception:
            embed_id = None
        if embed_id is not None:
            m = (input_ids == embed_id)
            if m.any():
                idx = m.float().argmax(dim=1)
                has_any = m.any(dim=1)
                out = last_hidden_state[:, 0, :].clone()
                b_idx = torch.arange(last_hidden_state.size(0), device=last_hidden_state.device)
                out[has_any] = last_hidden_state[b_idx[has_any], idx[has_any], :]
                return out

    return mean_pooling(last_hidden_state, attention_mask)


In [ ]:
# projection layer teacher -> student
# infer dims
d_s = model_student.config.hidden_size
d_t = model_teacher.config.hidden_size

# define matryoshka dims now that d_s is known
matryoshka_dims = [128, 256, 384, 512, d_s]
matryoshka_dims = [d for d in matryoshka_dims if d <= d_s]

# create projection if missing / overwritten
if ("proj_t2s" not in globals()) or (proj_t2s is None) or (not isinstance(proj_t2s, nn.Module)):
    print("[Fix] Initializing proj_t2s (teacher->student projection).")
    proj_t2s = nn.Linear(d_t, d_s, bias=False).to(device_s)

def _optimizer_has_param(opt, p):
    pid = id(p)
    for g in opt.param_groups:
        for q in g.get("params", []):
            if id(q) == pid:
                return True
    return False

# ensure optimizer includes proj_t2s params
if "optimizer" in globals():
    missing = [p for p in proj_t2s.parameters() if not _optimizer_has_param(optimizer, p)]
    if len(missing) > 0:
        print(f"[Fix] Adding proj_t2s params to optimizer (missing {len(missing)} tensors).")
        optimizer.add_param_group({"params": proj_t2s.parameters(), "lr": LR})
else:
    raise RuntimeError("optimizer not found. Run the optimizer/scheduler cell before this one.")

In [ ]:
# TRAINING LOOP 
from torch.cuda.amp import autocast

global_step = 0

for epoch in range(epochs):
    model_student.train()
    total_loss, n_items = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in pbar:
        batch_s, batch_t = {}, {}
        for k, v in batch.items():
            if not torch.is_tensor(v):
                continue
            if k.endswith("_stu") or k == "labels":
                batch_s[k] = v.to(device_s, non_blocking=True)
            if k.endswith("_tea"):
                batch_t[k] = v.to(device_t, non_blocking=True)

        # scheduled weights
        beta_mrl = beta_mrl_max * ramp(global_step, mrl_start, mrl_ramp)
        alpha_attn = alpha_attn_max * ramp(global_step, att_start, att_ramp)

        need_att = (alpha_attn > 0) and (att_every > 0) and (global_step % att_every == 0)

        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=torch.cuda.is_available()):
            # ================== TEACHER (device_t, no grad) ==================
            with torch.inference_mode():
                t_out1 = model_teacher(
                    input_ids=batch_t["input_ids1_tea"],
                    attention_mask=batch_t["attention_mask1_tea"],
                    output_attentions=need_att,
                    return_dict=True,
                )
                T_last1 = t_out1.last_hidden_state  # [B, L_t, d_t]

                T_sent = extract_teacher_sentence_embedding(
                    last_hidden_state=T_last1,
                    input_ids=batch_t["input_ids1_tea"],
                    attention_mask=batch_t["attention_mask1_tea"],
                    tokenizer=tok_teacher,
                    embed_token=teacher_special,   # "<|embed|>" by default
                )  # [B, d_t]

                # move sentence embedding only
                T_sent_s = T_sent.to(device_s, non_blocking=True)
                T_emb_s = proj_t2s(T_sent_s)  # [B, d_s]

                if need_att:
                    T_atts = [att.to(device_s, non_blocking=True) for att in t_out1.attentions]
                else:
                    T_atts = None

            # ================== STUDENT (device_s) ==================
            s_out1 = model_student(
                input_ids=batch_s["input_ids1_stu"],
                attention_mask=batch_s["attention_mask1_stu"],
                output_attentions=need_att,
                return_dict=True,
            )
            s_out2 = model_student(
                input_ids=batch_s["input_ids2_stu"],
                attention_mask=batch_s["attention_mask2_stu"],
                return_dict=True,
            )

            S_last1 = s_out1.last_hidden_state
            S_last2 = s_out2.last_hidden_state

            S_emb1 = get_student_sentence_emb(S_last1, batch_s["attention_mask1_stu"], student_pool)
            S_emb2 = get_student_sentence_emb(S_last2, batch_s["attention_mask2_stu"], student_pool)

            # (A) Main SimCSE InfoNCE
            loss_task, _ = info_nce(S_emb1, S_emb2)

            # (B) MRL (Matryoshka) distillation
            mrl_1 = matryoshka_prefix_cosine_loss(
                student_emb=S_emb1,
                teacher_emb=T_emb_s,
                dims=matryoshka_dims,
                weight_mode=mrl_weight_mode,
            )
            mrl_2 = matryoshka_prefix_cosine_loss(
                student_emb=S_emb2,
                teacher_emb=T_emb_s,
                dims=matryoshka_dims,
                weight_mode=mrl_weight_mode,
            )
            mrl_loss = 0.5 * (mrl_1 + mrl_2)

            # (C) IRA/CKA attention alignment (optional)
            if need_att:
                S_atts = s_out1.attentions
                att_loss = compute_att_loss_2(
                    teacher_atts=T_atts,
                    student_atts=S_atts,
                    input_ids_tea=batch_t["input_ids1_tea"].to(device_s, non_blocking=True),
                    input_ids_stu=batch_s["input_ids1_stu"],
                    attention_mask_tea=batch_t["attention_mask1_tea"].to(device_s, non_blocking=True),
                    attention_mask_stu=batch_s["attention_mask1_stu"],
                    tok_teacher=tok_teacher,
                    tok_student=tok_student,
                    k=k_ira,
                    device=device_s,
                    teacher_special=teacher_special,
                    student_special=student_special,
                    top_frac=top_frac,
                    min_tokens=min_tokens,
                    offset_mapping_tea=(batch_t["offset_mapping1_tea"].to(device_s, non_blocking=True)
                                       if "offset_mapping1_tea" in batch_t else None),
                    offset_mapping_stu=(batch_s["offset_mapping1_stu"]
                                       if "offset_mapping1_stu" in batch_s else None),
                )
            else:
                att_loss = torch.tensor(0.0, device=device_s)

            kd_sum = alpha_attn * att_loss + beta_mrl * mrl_loss

            # Total loss
            loss = (w_task * loss_task) + (alpha_kd * kd_sum)
            loss = loss.float()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # logging
        bs = next(iter(batch_s.values())).size(0)
        total_loss += loss.item() * bs
        n_items += bs
        avg_loss = total_loss / max(1, n_items)

        pbar.set_postfix({
            "avg": f"{avg_loss:.4f}",
            "task": f"{loss_task.detach().float().item():.3f}",
            "mrl": f"{mrl_loss.detach().float().item():.3f}",
            "att": f"{att_loss.detach().float().item():.3f}",
            "β_mrl": f"{beta_mrl:.2f}",
            "α_att": f"{alpha_attn:.3f}",
            "att?": int(need_att),
        })

        global_step += 1

        # cleanup
        del S_last1, S_last2, S_emb1, S_emb2, T_last1, T_sent, T_sent_s, T_emb_s, mrl_1, mrl_2, mrl_loss, loss
        if need_att:
            del T_atts, S_atts
        torch.cuda.empty_cache()

    # eval hooks
    eval_classification_task(model_student, test_cls_tasks)
    eval_pair_task(model_student, test_pair_tasks)
    eval_sts_task(model_student, test_sts_tasks)
